In [1]:
# custom_objects downloading
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
import numpy as np

@tf.keras.utils.register_keras_serializable()
class RepeatChannels(Layer):
    def __init__(self, **kwargs):
        super(RepeatChannels, self).__init__(**kwargs)

    def call(self, inputs):
        # Assuming the repeat operation repeats across channels
        return K.repeat_elements(inputs, rep=3, axis=-1)

    def get_config(self):
        config = super(RepeatChannels, self).get_config()
        return config

@tf.keras.utils.register_keras_serializable()
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = K.sum(y_true * y_pred)
        union = K.sum(y_true) + K.sum(y_pred) - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

@tf.keras.utils.register_keras_serializable()
def dice_coef(y_true, y_pred, smooth=1e-6):
    """
    Compute the Dice coefficient, a measure of overlap between two samples.
    """
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(y_true * y_pred)
    dice = (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    return dice

@tf.keras.utils.register_keras_serializable()
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

@tf.keras.utils.register_keras_serializable()
def bce_dice_loss(y_true, y_pred):
    # Binary Cross-Entropy
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    # Dice Loss
    intersection = tf.reduce_sum(y_true * y_pred)
    dice_loss = 1 - (2. * intersection + 1) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1)
    # Combine both losses
    return bce_loss + dice_loss

# Load models with custom objects
from tensorflow.keras.models import load_model

custom_objects = {
    'RepeatChannels': RepeatChannels,
    'iou': iou,
    'dice_coef': dice_coef,
    'dice_loss': dice_loss,
    'bce_dice_loss': bce_dice_loss
}

/Users/ammaster10/Documents/Github/Year4/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
DeepLabV3 = "/Users/ammaster10/Documents/Github/Year4/CNVresearch/Models/deeplabv3_plus_ultra_model_2025_02_10_01_37_48.keras"

In [26]:
def preprocess_noise(image, method='median', kernel_size=3):
    if kernel_size % 2 == 0 or kernel_size < 1:
        raise ValueError("Kernel size must be an odd integer greater than 1.")

    if method == 'gaussian':
        # Apply Gaussian Blur
        denoised = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    elif method == 'median':
        # Apply Median Filter
        denoised = cv2.medianBlur(image, kernel_size)
    elif method == 'bilateral':
        # Apply Bilateral Filter
        denoised = cv2.bilateralFilter(image, kernel_size, 75, 75)
    else:
        raise ValueError("Unsupported denoising method. Use 'gaussian', 'median', or 'bilateral'.")

    return denoised


In [27]:

def parse_coco_json(json_file, image_dir):
    """
    Parses the COCO JSON file and returns a dictionary of image masks.

    Args:
        json_file: Path to the COCO JSON file.
        image_dir: Directory containing the images.

    Returns:
        Dictionary with image filenames as keys and corresponding masks as values.
    """
    import json
    from pycocotools import mask as maskUtils
    from PIL import ImageDraw

    with open(json_file, 'r') as f:
        coco_data = json.load(f)

    annotations = {}
    image_data = {img['id']: img for img in coco_data['images']}

    for ann in coco_data['annotations']:
        img_info = image_data[ann['image_id']]
        img_filename = img_info['file_name']
        width, height = img_info['width'], img_info['height']

        if 'segmentation' in ann:
            segmentation = ann['segmentation']
            mask = segmentation_to_mask(segmentation, width, height)
            resized_mask = np.array(Image.fromarray(mask).resize((256, 256), Image.NEAREST))
            annotations[img_filename] = resized_mask

    return annotations

def segmentation_to_mask(segmentation, width, height):
    """
    Converts COCO segmentation to a binary mask.

    Args:
        segmentation: COCO segmentation data (RLE or polygon).
        width: Image width.
        height: Image height.

    Returns:
        Binary mask as a numpy array.
    """
    from pycocotools import mask as maskUtils
    from PIL import Image, ImageDraw

    if isinstance(segmentation, list):  # Polygon format
        mask = np.zeros((height, width), dtype=np.uint8)
        for polygon in segmentation:
            poly = np.array(polygon).reshape((-1, 2))
            img = Image.new('L', (width, height), 0)
            ImageDraw.Draw(img).polygon([tuple(p) for p in poly], outline=1, fill=1)
            mask = np.array(img, dtype=np.uint8)
        return mask
    elif isinstance(segmentation, dict):  # RLE format
        rle = maskUtils.frPyObjects(segmentation, height, width)
        mask = maskUtils.decode(rle)
        return mask
    return np.zeros((height, width), dtype=np.uint8)

In [5]:
import cv2
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt

# Load the DeepLabV3+ model with custom objects
model = load_model(DeepLabV3, custom_objects=custom_objects)

# Print model summary to verify it loaded correctly
print(f"Model loaded successfully from: {DeepLabV3}")
# model.summary()

Model loaded successfully from: /Users/ammaster10/Documents/Github/Year4/CNVresearch/Models/deeplabv3_plus_ultra_model_2025_02_10_01_37_48.keras


In [33]:
import os
import cv2  # You might need to run: pip install opencv-python
import numpy as np
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model # To load your model
# The following imports are for the COCO functions, which are included but not used in the main loop
import json
from pycocotools import mask as maskUtils
from PIL import ImageDraw
from pycocotools.coco import COCO

def run_inference_and_save_predictions(model, image_dir, coco_json_path, output_dir, target_size=(256, 256)):
    os.makedirs(output_dir, exist_ok=True)
    coco = COCO(coco_json_path)
    img_ids = coco.getImgIds()
    print(f"Found {len(img_ids)} image entries in the JSON. Processing with target_size={target_size}...")

    for img_id in tqdm(img_ids, desc="Predicting masks"):
        img_info = coco.loadImgs(img_id)[0]
        filename = img_info['file_name']

        image_path = os.path.join(image_dir, filename)

        try:
            if not os.path.exists(image_path):
                print(f"\nWarning: File not found, skipping -> {image_path}")
                continue
            
            # This will now use the corrected target_size (e.g., 512x512)
            img = load_img(image_path, color_mode='grayscale', target_size=target_size)
            
            img_array = img_to_array(img)
            img_uint8 = img_array.astype(np.uint8)
            img_denoised = preprocess_noise(img_uint8, method='median', kernel_size=3)
            img_normalized = img_denoised.astype(np.float32) / 255.0
            model_input = np.expand_dims(img_normalized, axis=0)

            predicted_mask = model.predict(model_input, verbose=0)[0]
            binary_mask = (predicted_mask > 0.5).astype(np.uint8)
            output_image_array = np.squeeze(binary_mask) * 255
            output_image = Image.fromarray(output_image_array, mode='L')
            
            base_name = os.path.splitext(filename)[0]
            output_path = os.path.join(output_dir, f"{base_name}_predicted_mask.png")
            output_image.save(output_path)

        except Exception as e:
            print(f"\nError processing {filename}: {e}")

    print(f"\n✅ Done! All predicted masks saved to: {output_dir}")


In [34]:
image_dir = "/Users/ammaster10/Documents/Github/Year4/CNVresearch/Validation/340_Folder/340_Test"
coco_json_path = "/Users/ammaster10/Documents/Github/Year4/CNVresearch/Validation/340_Folder/subset_340.json"
output_dir = "/Users/ammaster10/Documents/Github/Year4/CNVresearch/Validation/340_Folder/GVF_Ready"

run_inference_and_save_predictions(model, image_dir,coco_json_path=coco_json_path, output_dir=output_dir, target_size=(512, 512))

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Found 340 image entries in the JSON. Processing with target_size=(512, 512)...


Predicting masks:   0%|          | 0/340 [00:00<?, ?it/s]2025-08-16 20:42:04.916741: I external/local_xla/xla/service/service.cc:163] XLA service 0x31390d680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-08-16 20:42:04.916760: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): Host, Default Version
2025-08-16 20:42:05.001688: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1755351725.743988 1102705 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
/var/folders/q3/rnsm0fg975ld64pvb5fc5dmm0000gn/T/ipykernel_79864/1375145498.py:43: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  output_image = Image.fromarray(output_image_array, mode='L')
Predicting masks: 100%|██████████| 340/340


✅ Done! All predicted masks saved to: /Users/ammaster10/Documents/Github/Year4/CNVresearch/Validation/340_Folder/GVF_Ready


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, img_as_float
from skimage.filters import gaussian
from skimage.segmentation import find_boundaries
from morphsnakes import morphsnakes

# 1. Load original grayscale image and corresponding predicted mask
original = img_as_float(io.imread("OCT_image.png", as_gray=True))  # Your original OCT slice
mask = io.imread("DeepLabV3pp_mask.png", as_gray=True) > 0.5       # Binary mask

# 2. Smooth original image (optional but helps stability)
smoothed = gaussian(original, sigma=1.0)

# 3. Initialize the level set (phi) from the mask
init_ls = mask.astype(float)

# 4. Apply Morphological GVF Snake
mgvf = morphsnakes.MorphGAC(
    smoothed,
    smoothing=2,
    threshold=0.3,
    balloon=-1
)

mgvf.levelset = init_ls
for i in range(100):  # You can tune iterations
    mgvf.step()

# 5. Final mask
refined_mask = mgvf.levelset > 0.5

# 6. Visualization
plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.imshow(original, cmap='gray')
plt.title("Original")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(mask, cmap='gray')
plt.title("Initial Mask")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(refined_mask, cmap='gray')
plt.title("GVF Refined Mask")
plt.axis('off')

plt.tight_layout()
plt.show()